In [79]:
!pip install bson

  Created wheel for bson: filename=bson-0.5.10-cp37-none-any.whl size=11980 sha256=eff90a3fb3b183b5687a922a0a7187af7df5a4ef8fdca08ed9cdc29d94ad8a89
  Stored in directory: /Users/carlyfabris/Library/Caches/pip/wheels/19/e9/ab/932515e84e1300ea515cb4ae3297a5794a6ebbaeb649d50a3b
Successfully built bson


In [68]:
import json
import pandas as pd
import pymongo
import flask
import csv
import urllib

In [69]:
#with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    #counties = json.load(response)
    
import urllib.request
with urllib.request.urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
   geojson = json.load(response)

In [70]:
geojson['features'][0]

{'type': 'Feature',
 'properties': {'GEO_ID': '0500000US01001',
  'STATE': '01',
  'COUNTY': '001',
  'NAME': 'Autauga',
  'LSAD': 'County',
  'CENSUSAREA': 594.436},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-86.496774, 32.344437],
    [-86.717897, 32.402814],
    [-86.814912, 32.340803],
    [-86.890581, 32.502974],
    [-86.917595, 32.664169],
    [-86.71339, 32.661732],
    [-86.714219, 32.705694],
    [-86.413116, 32.707386],
    [-86.411172, 32.409937],
    [-86.496774, 32.344437]]]},
 'id': '01001'}

In [71]:
f = open('nyt_county_json_latest.json',) 
# returns JSON object as  
# a dictionary 
data = json.load(f)

In [72]:
data[0]

{'County': 'Weston',
 'state': 'Wyoming',
 'fips': 56045,
 'cases': 5,
 'deaths': 0,
 'death_pop_percent': 0.0,
 'death_case_percent': 0.0,
 'case_pop_percent': 0.0702543206,
 'death_pop_per_10000': 0.0,
 'death_case_per_10000': 0.0,
 'case_pop_per_10000': 7.0254320641}

In [73]:
for i in range(len(geojson['features'])):
    

    for j in range(len(data)):
        if data[j]['fips'] == int(geojson['features'][i]['id']):
            geojson['features'][i]['properties']['death_pop_percent'] = data[j]['death_pop_percent']
            geojson['features'][i]['properties']['death_case_percent'] = data[j]['death_case_percent']
            geojson['features'][i]['properties']['case_pop_percent'] = data[j]['case_pop_percent']
            geojson['features'][i]['properties']['death_pop_per_10000'] = data[j]['death_pop_per_10000']
            geojson['features'][i]['properties']['case_pop_per_10000'] = data[j]['case_pop_per_10000']
            geojson['features'][i]['properties']['death_case_per_10000'] = data[j]['death_case_per_10000']
            geojson['features'][i]['properties']['State_Name'] = data[j]['state']



In [74]:
geojson['features'][2]

{'type': 'Feature',
 'properties': {'GEO_ID': '0500000US01017',
  'STATE': '01',
  'COUNTY': '017',
  'NAME': 'Chambers',
  'LSAD': 'County',
  'CENSUSAREA': 596.531,
  'death_pop_percent': 0.1121109308,
  'death_case_percent': 4.4811320755,
  'case_pop_percent': 2.5018439298,
  'death_pop_per_10000': 11.2110930816,
  'case_pop_per_10000': 250.1843929783,
  'death_case_per_10000': 448.1132075472,
  'State_Name': 'Alabama'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-85.184131, 32.870525],
    [-85.123421, 32.772248],
    [-85.13204, 32.764211],
    [-85.136776, 32.746512],
    [-85.285043, 32.730731],
    [-85.593151, 32.72853],
    [-85.593177, 33.10734],
    [-85.232378, 33.108077],
    [-85.223261, 33.06258],
    [-85.221868, 33.055538],
    [-85.184131, 32.870525]]]},
 'id': '01017'}

In [44]:
# with open('final_geojson.json', 'w') as f:
#     json.dump(geojson, f, indent=4)

In [75]:
for element in geojson['features']:
    if 'id' in element:
        del element['id']
with open('final_geojson.json', 'w') as data_file:
    data = json.dump(geojson, data_file)

In [77]:
from flask_pymongo import PyMongo
from config import mongoURL 

from pymongo import MongoClient
import json
#client = MongoClient('localhost', 27017)
client = MongoClient(mongoURL)

In [81]:
 geojson_db = client['geojson']
collection_county_geojson = geojson_db['county_geojson']
with open('final_geojson.json') as f:
    final_geojson = json.load(f)
collection_county_geojson.insert_one(final_geojson)